In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:

import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout,Conv1D,MaxPooling1D # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [ ]:

english_stops = stopwords.words('english')


In [ ]:
def load_dataset():
    df = pd.read_csv('/content/train.csv',header=None, nrows=100000)
    df.columns = ['Polarity', 'Title', 'Review']
    df = df[['Polarity', 'Review']].reset_index(drop=True)
    x_data = df['Review']       # Reviews/Input
    y_data = df['Polarity']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # #ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace(1, 0)
    y_data = y_data.replace(2, 1)


    return x_data, y_data

x_data, y_data = load_dataset()



Review
0        [this, sound, track, beautiful, it, paints, se...
1        [i, reading, lot, reviews, saying, best, game,...
2        [this, soundtrack, favorite, music, time, hand...
3        [i, truly, like, soundtrack, i, enjoy, video, ...
4        [if, played, game, know, divine, music, every,...
                               ...                        
99995    [good, excelent, fantastic, wonderful, muy, bu...
99996    [these, are, the, real, adventures, of, the, r...
99997    [sylvain, beauregard, widely, known, die, hard...
99998    [i, think, i, know, person, wrote, book, i, ce...
99999    [this, call, overkill, how, many, books, need,...
Name: Review, Length: 100000, dtype: object 

Sentiment
0        1
1        1
2        1
3        1
4        1
        ..
99995    1
99996    0
99997    0
99998    0
99999    0
Name: Polarity, Length: 100000, dtype: int64


In [ ]:

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [this, sound, track, beautiful, it, paints, se...
1        [i, reading, lot, reviews, saying, best, game,...
2        [this, soundtrack, favorite, music, time, hand...
3        [i, truly, like, soundtrack, i, enjoy, video, ...
4        [if, played, game, know, divine, music, every,...
                               ...                        
99995    [good, excelent, fantastic, wonderful, muy, bu...
99996    [these, are, the, real, adventures, of, the, r...
99997    [sylvain, beauregard, widely, known, die, hard...
99998    [i, think, i, know, person, wrote, book, i, ce...
99999    [this, call, overkill, how, many, books, need,...
Name: Review, Length: 100000, dtype: object 

Sentiment
0        1
1        1
2        1
3        1
4        1
        ..
99995    1
99996    0
99997    0
99998    0
99999    0
Name: Polarity, Length: 100000, dtype: int64


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
59835    [the, first, couple, books, series, keep, atte...
38528    [i, got, next, day, i, ordered, used, standard...
25631    [would, loved, the, movie, stupid, boring, plu...
3077     [i, mean, hours, really, the, story, line, sim...
21221    [i, sorry, mr, lewis, writing, struck, pedanti...
                               ...                        
41145    [i, picked, high, wizardry, third, book, serie...
52114    [lacked, ability, keep, interest, story, commo...
42102    [finally, movie, average, man, relate, for, us...
88444    [a, movie, right, start, fails, finish, proper...
52443    [although, game, came, xbox, i, played, playst...
Name: Review, Length: 80000, dtype: object 

56655    [a, serious, informative, introduction, murky,...
68397    [i, know, book, one, classics, missed, piece, ...
19577    [this, is, one, book, that, i, knew, that, i, ...
18900    [bland, rip, great, comic, book, puts, worse, ...
41185    [i, always, smantha, james, lisa, kleypas, cat...
 

In [ ]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [ ]:
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[    2    17   326 ...  4159   657   398]
 [    1    57   192 ...     0     0     0]
 [   12   159     2 ...     0     0     0]
 ...
 [  349     8   937 ...  3969   508 10672]
 [   30     8   117 ...     0     0     0]
 [  224   135   183 ...   734     7 46284]] 

Encoded X Test
 [[   30   727  1499 ...     0     0     0]
 [    1    38     3 ...     1    71  1555]
 [    5   287     4 ...   612   137    79]
 ...
 [    1    28    76 ...  1712 20201     4]
 [    1  1076  2096 ...   152   203   320]
 [  125   780  1411 ...     0     0     0]] 

Maximum review length:  44


In [ ]:
EMBED_DIM = 32
LSTM_OUT = 64
FILTER_SIZE = 3
NUM_FILTERS = 32
POOL_SIZE = 2

model = Sequential()

# Add a CNN layer
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(Conv1D(filters = 64, kernel_size = 3, strides= 1, padding='same', activation= 'relu'))
model.add(MaxPooling1D(pool_size=POOL_SIZE))
model.add(Dropout(0.2))

# Add the LSTM layer
model.add(LSTM(LSTM_OUT))
model.add(Dropout(0.2))

# Add the output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 44, 32)            2883136   
                                                                 
 conv1d_1 (Conv1D)           (None, 44, 64)            6208      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 22, 64)            0         
 g1D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 22, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                      

In [ ]:
model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size = 128, epochs = 3)

Epoch 1/5
625/625 [==============================] - 73s 113ms/step - loss: 0.4106 - accuracy: 0.8027 - val_loss: 0.3410 - val_accuracy: 0.8526
Epoch 2/5
625/625 [==============================] - 68s 108ms/step - loss: 0.2579 - accuracy: 0.8979 - val_loss: 0.3514 - val_accuracy: 0.8504
Epoch 3/5
625/625 [==============================] - 67s 108ms/step - loss: 0.1730 - accuracy: 0.9361 - val_loss: 0.4147 - val_accuracy: 0.8413
Epoch 4/5
625/625 [==============================] - 75s 121ms/step - loss: 0.1072 - accuracy: 0.9630 - val_loss: 0.5271 - val_accuracy: 0.8324
Epoch 5/5
625/625 [==============================] - 72s 115ms/step - loss: 0.0640 - accuracy: 0.9789 - val_loss: 0.6524 - val_accuracy: 0.8252


In [ ]:
y_pred =  (model.predict(x_test) > 0.7).astype("int32")

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

625/625 [==============================] - 7s 10ms/step
Correct Prediction: 16461
Wrong Prediction: 3539
Accuracy: 82.30499999999999


In [ ]:
model.save('CNN_LSTM_AmazonReview_T1.keras')

In [ ]:
loaded_model = load_model('CNN_LSTM_AmazonReview_T1.keras')

In [ ]:
y_pred_1 =  (loaded_model.predict(x_test) > 0.5).astype("int32")

true = 0
for i, y in enumerate(y_test):
    if y == y_pred_1[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred_1) - true))
print('Accuracy: {}'.format(true/len(y_pred_1)*100))

625/625 [==============================] - 5s 7ms/step
Correct Prediction: 16503
Wrong Prediction: 3497
Accuracy: 82.515
